In [1]:
import os
import pandas as pd;pd.__version__

'0.24.2'

In [38]:
data_path = r'E:\Dropbox\Trade\MarketData\700base\NQ ##-##.csv'
trade_path = r'E:\Dropbox\Trade\MarketData\700base\NQ_2.txt'

In [39]:
trade_df=pd.read_csv(trade_path, parse_dates=['EntryDateTime', 'ExitDateTime'])
trade_df['Direction'] = trade_df['Direction'].str.replace('TR Entry ', '')

In [40]:
trade_df.head()

,Ticker,EntryDateTime,Direction,EntryReason,EntryPrice,InitialStopLoss,InitialProfitTarget,TimeFrame,ExitDateTime,PreventStopLoss,ProfitTargetCondition,ExitPrice,ExitReason
0,NQ ##-##,2018-09-26 11:28:37,Long,BG_BarFirstTrade,7638.00,7627.503571,7674.183913,25,2018-09-26 12:42:00,7627.503571,BackgroundVolatilityTarget,7627.503571,StopLossHit
1,NQ ##-##,2018-09-26 13:00:20,Long,BG_BarFirstTrade,7638.50,7624.914286,7674.375000,25,2018-09-26 14:47:30,7639.101661,FilterVolatilityTarget,7639.101661,StopLossHit
2,NQ ##-##,2018-09-26 15:54:54,Short,BG_BarReEntryTrade,7589.25,7597.982143,7568.318050,25,2018-09-26 15:55:06,7597.982143,FilterBO_Target,7592.250000,BG40%Fade
3,NQ ##-##,2018-09-27 10:05:04,Long,BG_BarFirstTrade,7668.50,7660.604888,7715.611324,25,2018-09-27 12:29:36,7660.604888,FilterInnerZone,7677.250000,ProfitTargetHit
4,NQ ##-##,2018-10-01 00:37:03,Long,BG_BarFirstTrade,7705.50,7695.210714,7747.687500,25,2018-10-01 08:38:53,7695.210714,BackgroundVolatilityTarget,7695.210714,StopLossHit


In [41]:
df = pd.read_csv(data_path, parse_dates=['TickDateTime']).set_index('TickDateTime')
df = df.loc[~df.index.duplicated(keep='first')]

In [42]:
def get_min_max_mae_of_bar(entry_time, exit_time, df):
    return df.loc[trade_entry_time:trade_exit_time,'HighPrice'].max(), df.loc[trade_entry_time:trade_exit_time,'LowPrice'].min()

In [43]:
for row_index, row in trade_df.iterrows():
    trade_entry_time = row['EntryDateTime']
    trade_exit_time = row['ExitDateTime']

    try:
        df.loc[trade_entry_time]
    except KeyError:
        print('Unable to find entry time, use the next available')
        trade_entry_time = df.loc[trade_entry_time:].index[0]
#     print(f'{trade_entry_time}, {row_index}')

    try:
        df.loc[trade_exit_time]
    except KeyError:
        print('Unable to find exit time, use the next available')
        trade_exit_time = df.loc[trade_exit_time:].index[0]

    bg_bar_number_name = 'BgHtfBarNumber'
    bg_entry_bar_number = df.loc[trade_entry_time, bg_bar_number_name]
    bg_exit_bar_number = df.loc[trade_exit_time, bg_bar_number_name]
    
    max_value, min_value = get_min_max_mae_of_bar(trade_entry_time, trade_exit_time, df)
    
    if row['Direction'] == 'Long':
        trade_df.loc[row_index, 'MAE'] = min_value
        trade_df.loc[row_index, 'MFE'] = max_value
    else:
        trade_df.loc[row_index, 'MAE'] = max_value
        trade_df.loc[row_index, 'MFE'] = min_value
    
    try:
        for i in range(0,5):
            _entry_index = df[df[bg_bar_number_name] == bg_entry_bar_number + i].index[-1] 
            max_value, min_value = get_min_max_mae_of_bar(trade_entry_time, _entry_index, df)
            trade_df.loc[row_index, f'EntryBar{i+1}Price'] = df.loc[_entry_index, 'ClosePrice']
            max_value, min_value = get_min_max_mae_of_bar(trade_entry_time, _entry_index, df)
            
            if row['Direction'] == 'Long':
                trade_df.loc[row_index, f'P{i+1}_EntryMAE'] = min_value
                trade_df.loc[row_index, f'P{i+1}_EntryMFE'] = max_value
            else:
                trade_df.loc[row_index, f'P{i+1}_EntryMAE'] = max_value
                trade_df.loc[row_index, f'P{i+1}_EntryMFE'] = min_value
        
        for _i in range(0,5):
            _exit_index = df[df[bg_bar_number_name] == bg_exit_bar_number + i].index[-1] 
            max_value, min_value = get_min_max_mae_of_bar(bg_exit_bar_number, _exit_index, df)
            trade_df.loc[row_index, f'ExitBar{i+1}Price'] = df.loc[_exit_index, 'ClosePrice']
            max_value, min_value = get_min_max_mae_of_bar(trade_exit_time, _exit_index, df)
            if row['Direction'] == 'Long':
                trade_df.loc[row_index, f'P{i+1}_ExitMAE'] = min_value
                trade_df.loc[row_index, f'P{i+1}_ExitMFE'] = max_value
            else:
                trade_df.loc[row_index, f'P{i+1}_ExitMAE'] = max_value
                trade_df.loc[row_index, f'P{i+1}_ExitMFE'] = min_value
    except IndexError:
        pass

    fetch_columns = ['BgAtr', 'BgVWAP', 'BgVWAPPrior', 'BgYellow', 'BgGreen', 'BgGreenPrior','BgVPOC', 'BgVPOCPrior', 'BgTrendState', 'BgTwoPeriodHigh', 'BgTwoPeriodLow', 'BgValueArea', 'FtAtr', 'FtVWAP', 'FtVWAPPrior', 'FtYellow', 'FtGreen', 'FtGreenPrior', 'FtVPOC', 'FtVPOCPrior', 'FtTrendState', 'FtTwoPeriodHigh', 'FtTwoPeriodLow', 'FtValueArea']
    for _column in fetch_columns:
        trade_df.loc[row_index, _column] = df.loc[trade_entry_time, _column]


In [44]:
trade_processed_path = trade_path.replace('.txt', '_processed.csv')
trade_df.to_csv(trade_processed_path, index=False)